In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## PUBG Data Analysis 

In this kernal i am going to explore PUBG dataset,visualise the data and conclude something

<img src="https://cdn.images.express.co.uk/img/dynamic/143/590x/PUBG-886916.jpg">

## Let's Start

In [ ]:
import numpy as np
import pandas as pd
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Read Data
train = pd.read_csv('../input/train_V2.csv')
test = pd.read_csv('../input/test_V2.csv')

In [ ]:
train.info()

In [ ]:
train.columns

## FEATURE DESCRIPTION'S FROM KAGGLE 
*  DBNOs - Number of enemy players knocked.
*     assists - Number of enemy players this player damaged that were killed by teammates.
*     boosts - Number of boost items used.
*     damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
*     headshotKills - Number of enemy players killed with headshots.
*     heals - Number of healing items used.
*     Id - Player’s Id
*    killPlace - Ranking in match of number of enemy players killed.
*     killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value       
       other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
*     killStreaks - Max number of enemy players killed in a short amount of time.
*     kills - Number of enemy players killed.
*     longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a 
       player and driving away may lead to a large longestKill stat.
*     matchDuration - Duration of match in seconds.
*     matchId - ID to identify match. There are no matches that are in both the training and testing set.
*     matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “      
      solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
*  rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use
      with caution. Value of -1 takes place of “None”. 
*   revives - Number of times this player revived teammates.
* rideDistance - Total distance traveled in vehicles measured in meters.
*   roadKills - Number of kills while in a vehicle.
*   swimDistance - Total distance traveled by swimming measured in meters.
*   teamKills - Number of times this player killed a teammate.
*   vehicleDestroys - Number of vehicles destroyed.
*   walkDistance - Total distance traveled on foot measured in meters.
*  weaponsAcquired - Number of weapons picked up.
*  winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a
      value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
*  groupId - ID to identify a group within a match. If the same group of players plays in different matches, they will have a       
     different groupId each time.
*   numGroups - Number of groups we have data for in the match.
*   maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data   
    skips over placements.
* winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0
    corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing
    chunks in a match.


In [ ]:
train.head()

In [ ]:
# Check row with NaN value
train[train['winPlacePerc'].isnull()]

#### We have to predict "winPlacePerc" .So we have to delete 2744604 row where "winPlacePerc" is Nan shown above

In [ ]:
# Delete this player
train.drop(2744604, inplace=True)

## PLOTS
Number of player joined

In [ ]:
# playersJoined
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
plt.figure(figsize=(15,10))
sns.countplot(train[train['playersJoined']>=75]['playersJoined'])
plt.title('playersJoined')
plt.show()

Number of weapon acquired
Mostly players had less 20 weapon


In [ ]:
weapon = train[train['weaponsAcquired']<=20]
sns.countplot(x='weaponsAcquired',data =weapon)

### Heals and Boost
We create a feature called 'healsandboosts' by adding heals and boosts

In [ ]:
train['healsandboosts'] = train['heals'] + train['boosts']
train[['heals', 'boosts', 'healsandboosts']].tail()

### Killing without moving
We try to identify cheaters by checking if people are getting kills without moving. We first identify the totalDistance travelled by a player and then set a boolean value to True if someone got kills without moving a single inch. We will remove cheaters in our outlier detection section.


In [ ]:
train['totalDistance'] = train['rideDistance'] + train['walkDistance'] + train['swimDistance']
train['killsWithoutMoving'] = ((train['kills'] > 0) & (train['totalDistance'] == 0))



In [ ]:
display(train[train['killsWithoutMoving'] == True].shape)
train[train['killsWithoutMoving'] == True].head(10)

In [ ]:
# Remove 
train.drop(train[train['killsWithoutMoving'] == True].index, inplace=True)

### Players who got more than 10 roadKills


In [ ]:
train[train['roadKills'] > 10]

### There are some player who have more kills. Let's see them


In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(data=train, x=train['kills']).set_title('Kills')
plt.show()

### There are very few players having more then 10 kills , maximum kills = 72 

In [ ]:
# Players who got more than 30 kills
display(train[train['kills'] > 30].shape)
train[train['kills'] > 30].head(10)

### Remove outliers 
players having kills more than 30 (remove them)

In [ ]:
# Remove outliers
train.drop(train[train['kills'] > 30].index, inplace=True)

### Find more outliers

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['longestKill'], bins=10)
plt.show()

In [ ]:
display(train[train['longestKill'] >= 1000].shape)
train[train['longestKill'] >= 1000].head(10)

There is something fishy going on with these players. We are probably better off removing them from our dataset

### Let's check out anomalies in Distance travelled.

In [ ]:
train[['walkDistance', 'rideDistance', 'swimDistance', 'totalDistance']].describe()

walk distance


In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['walkDistance'], bins=10)
plt.show()

In [ ]:
train.drop(train[train['walkDistance'] >= 10000].index, inplace=True)

rideDistance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['rideDistance'], bins=10)
plt.show()

In [ ]:
train.drop(train[train['rideDistance'] >= 20000].index, inplace=True)

swimDistance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['swimDistance'], bins=10)
plt.show()

In [ ]:
train[train['swimDistance'] >= 2000]

In [ ]:
# Remove outliers
train.drop(train[train['swimDistance'] >= 2000].index, inplace=True)

### weaponsAcquired

Most people acquire between 0 and 10 weapons in a game, but you also see some people acquire more than 80 weapons! Let's check these guys ou

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['weaponsAcquired'], bins=100)
plt.show()

In [ ]:
display(train[train['weaponsAcquired'] >= 80].shape)
train[train['weaponsAcquired'] >= 80].head()

In [ ]:
# Remove outliers
train.drop(train[train['weaponsAcquired'] >= 80].index, inplace=True)

### We have successfully removed around 1500 players 